In [2]:
import pandas as pd
import nltk


In [3]:
books_data = pd.read_csv('./cleaned_data/cleaned_books.csv')
books_data.shape

(141254, 5)

In [4]:
ratings_data = pd.read_csv('./cleaned_data/cleaned_ratings.csv')
ratings_data

,title,price,review_helpfulness,review_score,summary,text
0,Its Only Art If Its Well Hung!,NaN,7/7,4.0,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,NaN,10/10,5.0,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,NaN,10/11,5.0,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,NaN,7/7,4.0,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,NaN,3/3,4.0,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...
1623650,The Idea of History,NaN,12/14,5.0,A magnificent book if you're motivated enough,R. G. Collingwood's The Idea of History would ...
1623651,The Idea of History,NaN,3/3,4.0,Thoughtful Critic of History,History is not a scientific process of cutting...
1623652,The Idea of History,NaN,14/19,4.0,Difficult,"This is an extremely difficult book to digest,..."
1623653,The Idea of History,NaN,1/1,4.0,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...


In [7]:
# adapted from https://monkeylearn.com/blog/text-cleaning/
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
tqdm.pandas()

sub = r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?"
stemmer = PorterStemmer()
lemmer = WordNetLemmatizer()

stopwords = stopwords.words('english')

# Import caching
from functools import cache

@cache
def stem(word):
    return stemmer.stem(word)

def clean_text(sentence):
    #lowercase
    sentence = str(sentence)
    sentence = sentence.lower()

    #remove weird characters
    sentence = re.sub(sub, "",sentence)

    #remove stopwords
    # sentence = " ".join([word for word in sentence.split() if word not in (stopwords)])

    #stem words -> Try if a failure
    sentence = " ".join([stem(word) for word in sentence.split(" ") if word not in (stopwords)])
    return sentence


cleaned_text = ratings_data['text'].progress_map(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacksonbremen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
  0%|          | 0/1623655 [00:00<?, ?it/s]

100%|██████████| 1623655/1623655 [14:12<00:00, 1905.69it/s]


In [8]:
cleaned_text.to_csv("./cleaned_data/cleaned_text_stemmed.csv")

In [7]:
# Count nan values
cleaned_text.isna().sum()

0

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(ratings_data['text'])

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
vectorizer.vocabulary_

In [10]:
cleaned_text

0          julie strain fans collection photos 80 pages w...
1          dont care much dr seuss reading philip nels bo...
2          people become books read child father man dr s...
3          theodore seuss geisel 19041991 aka quotdr seus...
4          philip nel dr seuss american iconthis basicall...
                                 ...                        
2193369    extremely difficult book digest casual readers...
2193370    pretty interesting collingwood seems like firs...
2193371    good book esoteric history eh carr easier sele...
2193372    daughter freshman indiana university emailed l...
2193373    guy good ideas reader beware transciption seve...
Name: text, Length: 2193374, dtype: object